In [4]:
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pynput.keyboard import Key, Controller
import re
import sqlite3

conn = sqlite3.connect('data.db')
c = conn.cursor()

# Create technologies table
c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='techs' ''')
if c.fetchone()[0] == 0:
    c.execute('''CREATE TABLE techs(
            id varchar NOT NULL PRIMARY KEY,
            regex varchar NOT NULL,
            type varchar NOT NULL DEFAULT 'language'
        )
    ''')
    c.execute('''CREATE UNIQUE INDEX techs_unique_name ON techs(id)''')
    c.execute('''CREATE UNIQUE INDEX techs_unique_regex ON techs(regex)''')
else:
    print("Table techs exists")

# Create positions table
c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='positions' ''')
if c.fetchone()[0] == 0:
    c.execute('''CREATE TABLE positions(
            id integer NOT NULL PRIMARY KEY,
            name varchar NOT NULL,
            salary real NULL,
            company varchar,
            location varchar,
            source varchar NOT NULL
        )
    ''')
else:
    print("Table positions exists")

# Create linker
c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='positions_techs' ''')
if c.fetchone()[0] == 0:
    c.execute('''CREATE TABLE positions_techs(
            tech_id varchar,
            position_id integer,
            PRIMARY KEY (tech_id, position_id),
            FOREIGN KEY (tech_id) REFERENCES techs(id) ON DELETE CASCADE,
            FOREIGN KEY (position_id) REFERENCES positions(id) ON DELETE CASCADE
        )
    ''')
else:
    print("Table positions_techs exists")

# Add Programming Languages
c.execute('''INSERT OR IGNORE INTO techs (id, regex) VALUES
('Javascript', 'javascript'),
('Java', '[ ,]java[ \.,]'),
('C', '[ ,]c[ \.,]'),
('C++', 'c\+\+'),
('Kotlin', 'kotlin'),
('Swift', 'swift'),
('Ruby', 'ruby'),
('Python', 'python3?'),
('SQL', 'sql'),
('R', '[ ,]r[ \.,]'),
('Perl', 'perl'),
('C#', 'c#'),
('Lua', 'lua'),
('Rust', 'rust'),
('PHP', 'php'),
('HTML', 'html'),
('CSS', 'css'),
('XML', 'xml')
''')

c.execute('''INSERT OR IGNORE INTO techs (id, regex, type) VALUES
('React', 'react', 'framework'),
('Vue', 'vue', 'framework'),
('Express', 'express', 'framework'),
('Spring', 'spring', 'framework'),
('Django', 'django', 'framework'),
('Flask', 'flask', 'framework'),
('Angular', 'angular', 'framework'),
('Ruby on Rails', '(ruby on rails)|(rails)', 'framework'),
('ASP.NET Core', '(asp\.net core)|(asp\.net)', 'framework'),
('.NET', '[ ,]\.net[ ,\.]', 'framework'),
('jQuery', ' jquery', 'framework'),
('Laravel', 'laravel', 'framework')
''')

conn.commit()

def load_techs():
    c.row_factory = sqlite3.Row
    c.execute(''' SELECT * FROM techs ''')
    techs = {}
    for record in c.fetchall():
        techs[record['id']] = record['regex']
    return techs

technologies = load_techs()

Table techs exists
Table positions exists
Table positions_techs exists


In [5]:
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from pynput.keyboard import Key, Controller
import re

scraped_jobs = []
job_set = set()

# Initialize Driver
url = 'https://www.linkedin.com/jobs/search/?keywords=software%20engineer'
driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
driver.get(url)
keyboard = Controller()

# WHEN SELENIUM IS RUNNING, DON'T CLICK AROUND OTHER TABS, THIS WILL BREAK PROGRAM
def main():
    load_more_jobs = 0
    num_jobs_to_search = 25
    num_jobs_searched_so_far = 0
    load_more_jobs_text = 'infinite-scroller__show-more-button infinite-scroller__show-more-button--visible'

    #
    while num_jobs_searched_so_far < num_jobs_to_search:
        # Each job
        job_list = driver.find_elements_by_class_name('base-card__full-link')
        for i in range(num_jobs_searched_so_far, len(job_list)):
            job_list[i].click()
            sleep(3) # CHANGE THIS TO MAKE IT CLICK ON NEXT PAGE FASTER. CAREFUL IF THE PAGE DOESN'T LOAD THOUGH
            num_jobs_searched_so_far += 1

            # Try to load more jobs
            try:
                driver.find_element_by_xpath('/html/body/div[1]/div/main/section[2]/button').click() # Load More Jobs
                sleep(2)
            # If not possible, then get the current job
            except ElementNotInteractableException:
                try:
                    # Get salary stuff
                    num_jobs_searched_so_far += 1
                    company = driver.find_element_by_class_name('topcard__org-name-link').text
                    job_position = driver.find_element_by_class_name('top-card-layout__title').text
                    location = driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/section/div/div[1]/div/h4/div[1]/span[2]').text # [8:-1] gets location
                    salary = process_linkedin_salary(driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div[1]/section[1]/div/div/div').text)
                    job_description = driver.find_element_by_class_name('show-more-less-html__markup').text # Job Description
                    print("Company: " + company + ". Job Position: " + job_position + ". Location: " + location + ". Salary: " + str(salary))
                # Doesn't have salary
                except NoSuchElementException:
                    print("No Salary")
        # Load more jobs by pressing space
        keyboard.press(Key.space)
        keyboard.release(Key.space)
        sleep(2)
        keyboard.press(Key.space)
        keyboard.release(Key.space)
        sleep(2)

    print("Finished searching")
#     record_data(scraped_jobs)

# Helper function to extract programming languages from the job description text
def extract_languages(job_description):
    return

# Used to get the name of Job company without star ratings
def process_company(company):
    try:
        if driver.find_element_by_class_name('css-1m5m32b') != None:
            return company[0:-4]
    except NoSuchElementException:
        return company


# Function specific to LinkedIn, don't try to use for glassdoor
# $123,000.00/yr - $180,000.00/yr ==> Becomes avg
def process_linkedin_salary(salary):
    output = re.findall(r'\d+', salary)

    # Sometimes it falsly reads salaries. If there are no numbers, throw error
    if len(output) <= 3:
        raise NoSuchElementException('No Salary')
    # output is now [123, 000, 00, 180, 000, 00] from example

    # Output is sometimes 4 elements long, ex: $123,000/yr - 180,000/yr  --> No decimal places
    # Check if output has decimal places or not
    if len(output) == 6:
        # combine first two elements for min pay, combine 4th and 5th elements for max pay
        min_pay = int(output[0] + output[1])
        max_pay = int(output[3] + output[4])
        return int((min_pay + max_pay) / 2)
    # Or not -> output is [123, 000, 180, 000]
    min_pay = int(output[0] + output[1])
    max_pay = int(output[2] + output[3])
    return int((min_pay + max_pay) / 2)

# Format the data received from parsing a Job Card
def format_data(job_row):
    if job_row not in job_set:
        print(job_row)
        scraped_jobs.append(job_row)
        job_set.add(job_row)

# At the end, I want to record all my unique jobs to a CSV file
def record_data(records):
    with open('salaries.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Company','Job Title', 'Salary'])
        writer.writerows(records)

# Run my main program
main()


In [5]:
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from pynput.keyboard import Key, Controller
import re

scraped_jobs = []
job_set = set()

# Initialize Driver
url = 'https://www.linkedin.com/jobs/search/?keywords=software%20engineer'
driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
driver.get(url)
keyboard = Controller()

# WHEN SELENIUM IS RUNNING, DON'T CLICK AROUND OTHER TABS, THIS WILL BREAK PROGRAM
def main():
    load_more_jobs = 0
    num_jobs_to_search = 25
    num_jobs_searched_so_far = 0
    load_more_jobs_text = 'infinite-scroller__show-more-button infinite-scroller__show-more-button--visible'

    #
    while num_jobs_searched_so_far < num_jobs_to_search:
        # Each job
        job_list = driver.find_elements_by_class_name('base-card__full-link')
        for i in range(num_jobs_searched_so_far, len(job_list)):
            job_list[i].click()
            sleep(3) # CHANGE THIS TO MAKE IT CLICK ON NEXT PAGE FASTER. CAREFUL IF THE PAGE DOESN'T LOAD THOUGH
            num_jobs_searched_so_far += 1

            # Try to load more jobs
            try:
                driver.find_element_by_xpath('/html/body/div[1]/div/main/section[2]/button').click() # Load More Jobs
                sleep(2)
            # If not possible, then get the current job
            except ElementNotInteractableException:
                try:
                    # Get salary stuff
                    num_jobs_searched_so_far += 1
                    company = driver.find_element_by_class_name('topcard__org-name-link').text
                    job_position = driver.find_element_by_class_name('top-card-layout__title').text
                    location = driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/section/div/div[1]/div/h4/div[1]/span[2]').text # [8:-1] gets location
                    salary = process_linkedin_salary(driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div[1]/section[1]/div/div/div').text)
                    job_description = driver.find_element_by_class_name('show-more-less-html__markup').text # Job Description
                    print("Company: " + company + ". Job Position: " + job_position + ". Location: " + location + ". Salary: " + str(salary))
                # Doesn't have salary
                except NoSuchElementException:
                    print("No Salary")
        # Load more jobs by pressing space
        keyboard.press(Key.space)
        keyboard.release(Key.space)
        sleep(2)
        keyboard.press(Key.space)
        keyboard.release(Key.space)
        sleep(2)

    print("Finished searching")
#     record_data(scraped_jobs)

# Helper function to extract programming languages from the job description text
def extract_languages(job_description):
    return

# Used to get the name of Job company without star ratings
def process_company(company):
    try:
        if driver.find_element_by_class_name('css-1m5m32b') != None:
            return company[0:-4]
    except NoSuchElementException:
        return company


# Function specific to LinkedIn, don't try to use for glassdoor
# $123,000.00/yr - $180,000.00/yr ==> Becomes avg
def process_linkedin_salary(salary):
    output = re.findall(r'\d+', salary)

    # Sometimes it falsly reads salaries. If there are no numbers, throw error
    if len(output) <= 3:
        raise NoSuchElementException('No Salary')
    # output is now [123, 000, 00, 180, 000, 00] from example

    # Output is sometimes 4 elements long, ex: $123,000/yr - 180,000/yr  --> No decimal places
    # Check if output has decimal places or not
    if len(output) == 6:
        # combine first two elements for min pay, combine 4th and 5th elements for max pay
        min_pay = int(output[0] + output[1])
        max_pay = int(output[3] + output[4])
        return int((min_pay + max_pay) / 2)
    # Or not -> output is [123, 000, 180, 000]
    min_pay = int(output[0] + output[1])
    max_pay = int(output[2] + output[3])
    return int((min_pay + max_pay) / 2)

# Format the data received from parsing a Job Card
def format_data(job_row):
    if job_row not in job_set:
        print(job_row)
        scraped_jobs.append(job_row)
        job_set.add(job_row)

# At the end, I want to record all my unique jobs to a CSV file
def record_data(records):
    with open('salaries.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Company','Job Title', 'Salary'])
        writer.writerows(records)

# Run my main program
main()


In [6]:
# Clean up old entries from running script
SOURCE = 'glassdoor'
c.execute(f"DELETE FROM positions WHERE source='{SOURCE}'")

scraped_jobs = []
job_set = set()

# Initialize Driver
url = 'https://www.glassdoor.com/Job/software-engineer-jobs-SRCH_KO0,17.htm'
driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
driver.get(url)
keyboard = Controller()

# WHEN SELENIUM IS RUNNING, DON'T CLICK AROUND OTHER TABS, THIS WILL BREAK PROGRAM
def main():
    keyboard.press(Key.esc)
    keyboard.release(Key.esc)

    num_pages_to_search = 15
    num_pages_searched = 0
    num_jobs_searched = 0

    # extract the job data in "num_pages_to_search" pages
    while num_pages_searched < num_pages_to_search:
        # Each job
        jobs_on_page = driver.find_elements_by_class_name('react-job-listing')

        for job_card in jobs_on_page:
            # sleep to load the info
            count = 0
            sleep(0.75)
            while count < 12:
                try:
                    driver.find_element_by_class_name('jobDescriptionContent')
                    count += 1
                    break
                except Exception:
                    sleep(0.1)


            keyboard.press(Key.esc)
            keyboard.release(Key.esc)
            keyboard.press(Key.esc)
            keyboard.release(Key.esc)

            try:
                job_description = driver.find_element_by_class_name('jobDescriptionContent').text.lower() # THIS IS JOB DESCRIPTION
                techs = extract_technologies(job_description) # TODO: Extract Programming Languages from Job Description
                salary = process_salary(driver)
                company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
                job_position = driver.find_element_by_class_name('css-1j389vi').text
                location = driver.find_element_by_class_name('css-56kyx5').text
                save_data({'company': company, 'salary': salary, 'position': job_position, 'technologies': techs, 'location': location}, source=SOURCE) # Format information
            except Exception as ex:
                print(str(ex))

            job_card.click()

        sleep(1)
        keyboard.press(Key.esc)
        keyboard.release(Key.esc)
        keyboard.press(Key.esc)
        keyboard.release(Key.esc)

        try:
            job_description = driver.find_element_by_class_name('jobDescriptionContent').text.lower() # THIS IS JOB DESCRIPTION
            techs = extract_technologies(job_description) # TODO: Extract Programming Languages from Job Description
            salary = process_salary(driver)
            company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
            job_position = driver.find_element_by_class_name('css-1j389vi').text
            location = driver.find_element_by_class_name('css-56kyx5').text
            location = None if location == "" else location
            save_data({'company': company, 'salary': salary, 'position': job_position, 'technologies': techs, 'location': location}, source=SOURCE) # Format information
        except Exception as ex:
            print(str(ex))

        num_pages_searched += 1
        if num_pages_searched == num_pages_to_search:
            print("Finished Searching.")

        # go to next page
        driver.find_element_by_class_name('nextButton').click()
        sleep(5)

# Helper function to extract programming languages from the job description text
def extract_technologies(job_description):
    temp = set()
    for tech, reg in technologies.items():
        if re.search(reg, job_description, flags=re.IGNORECASE) is not None:
            temp.add(tech)
    return list(temp)

# Used to get the name of Job company without star ratings
def process_company(company):
    try:
        if driver.find_element_by_class_name('css-1m5m32b') is not None:
            return company[0:-4]
    except NoSuchElementException:
        return company

# $70,000 /yr (est.) => 70000
# $35 /hr (est.) => 35*52*40
def process_salary(driver):
    try:
        salary = driver.find_element_by_class_name('css-y2jiyn').text
        new_salary = salary.split(' ', 1)
        if new_salary[1] == '/yr (est.)':
            return float(re.sub("[^\d\.]", "", new_salary[0][1:]))
        else:
            return str(float(new_salary[0][1:]) * 52 * 40)
    except NoSuchElementException:
        return None

# Format the data received from parsing a Job Card
def format_data(job_row):
    if job_row not in job_set:
        scraped_jobs.append(job_row)
        job_set.add(job_row)

def save_data(data, source=''):
    position_query = "INSERT INTO positions (name, salary, company, location, source) VALUES (?, ?, ?, ?) RETURNING id"
    c.execute(position_query, (data['position'], data['salary'], data['company'], data['location'], source))
    new_id = c.fetchone()[0]
    linkers = []
    tech_query = "INSERT OR IGNORE INTO positions_techs (position_id, tech_id) VALUES (?, ?)"
    for tech in data['technologies']:
        linkers.append((new_id, tech))
    c.executemany(tech_query, linkers)
    conn.commit()

# Run my main program
main()




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/christiantingle/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_9864/1164290062.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_9864/1164290062.py:26: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  jobs_on_page = driver.find_elements_by_class_name('react-job-listing')
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_9864/1164290062.py:34: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver

Message: no such element: Unable to locate element: {"method":"css selector","selector":".jobDescriptionContent"}
  (Session info: chrome=101.0.4951.41)
Stacktrace:
0   chromedriver                        0x0000000100b949a0 chromedriver + 4426144
1   chromedriver                        0x0000000100b2e798 chromedriver + 4007832
2   chromedriver                        0x000000010078acc4 chromedriver + 191684
3   chromedriver                        0x00000001007b98d4 chromedriver + 383188
4   chromedriver                        0x00000001007e2648 chromedriver + 550472
5   chromedriver                        0x00000001007aecf4 chromedriver + 339188
6   chromedriver                        0x0000000100b6b7d0 chromedriver + 4257744
7   chromedriver                        0x0000000100b6f580 chromedriver + 4273536
8   chromedriver                        0x0000000100b736c4 chromedriver + 4290244
9   chromedriver                        0x0000000100b6fcf8 chromedriver + 4275448
10  chromedriver   

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=101.0.4951.41)
Stacktrace:
0   chromedriver                        0x0000000100b949a0 chromedriver + 4426144
1   chromedriver                        0x0000000100b2e798 chromedriver + 4007832
2   chromedriver                        0x000000010078acc4 chromedriver + 191684
3   chromedriver                        0x000000010078d7ac chromedriver + 202668
4   chromedriver                        0x000000010078d614 chromedriver + 202260
5   chromedriver                        0x000000010078d848 chromedriver + 202824
6   chromedriver                        0x00000001007bb018 chromedriver + 389144
7   chromedriver                        0x00000001007b0528 chromedriver + 345384
8   chromedriver                        0x00000001007afff4 chromedriver + 344052
9   chromedriver                        0x00000001007e2648 chromedriver + 550472
10  chromedriver          

In [13]:
# c.execute("SELECT * FROM positions WHERE salary IS NOT NULL")
# records = c.fetchall()
# print(len(records))
# for i in records:
#     print(i['salary'])
#     val = float(re.sub("[^\d\.]", "", f'{i["salary"]}'))
#     print(val)
#     c.execute(f"UPDATE positions SET salary={val} WHERE id={i['id']}")
# conn.commit()

246
145517.0
145517.0
145517.0
145517.0
102206.0
102206.0
106500.0
106500.0
77762.0
77762.0
82969.0
82969.0
82969.0
82969.0
195000.0
195000.0
83681.0
83681.0
98025.0
98025.0
106018.0
106018.0
106018.0
106018.0
127000.0
127000.0
172000.0
172000.0
134000.0
134000.0
112998.0
112998.0
117533.0
117533.0
162500.0
162500.0
102500.0
102500.0
104459.0
104459.0
129151.0
129151.0
72500.0
72500.0
112665.0
112665.0
270400.0
270400.0
266000.0
266000.0
140000.0
140000.0
72500.0
72500.0
90000.0
90000.0
41000.0
41000.0
75593.0
75593.0
75593.0
75593.0
145600.0
145600.0
135000.0
135000.0
118146.0
118146.0
162000.0
162000.0
115000.0
115000.0
161200.0
161200.0
95000.0
95000.0
100752.0
100752.0
110000.0
110000.0
97500.0
97500.0
105000.0
105000.0
160000.0
160000.0
162000.0
162000.0
118109.0
118109.0
110000.0
110000.0
110000.0
110000.0
134000.0
134000.0
138085.0
138085.0
85813.0
85813.0
68500.0
68500.0
90000.0
90000.0
105000.0
105000.0
168000.0
168000.0
168000.0
168000.0
135000.0
135000.0
106018.0
106018.0
10